In [1]:
# import numpy as np
import cv2
from matplotlib import pyplot as plt


def rescale_array(inputArr,minVal = 0,maxVal = 255):
    arrayMin = inputArr.min()
    arrayMax = inputArr.max()
    coeff = (maxVal-minVal)/(arrayMax-arrayMin)
    rescaledArray =  coeff*(inputArr-arrayMin) + minVal
    return rescaledArray

#equivalent to this 


In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

img = cv2.imread(r'C:\Users\Vardan\Documents\MATLAB\Viropath\CNN\water_coins.jpg')


gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

#noise removal
kernel = np.ones((1,1),np.uint8)
opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)


# sure background area  
sure_bg = cv2.dilate(opening,kernel,iterations=3)

# Finding sure foreground area
dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
ret, sure_fg = cv2.threshold(dist_transform,0.7*dist_transform.max(),255,0)


# Finding unknown region
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg,sure_fg)


# Marker labelling
ret, markers = cv2.connectedComponents(sure_fg)


# Add one to all labels so that sure background is not 0, but 1
markers = markers+1

# Now, mark the region of unknown with zero
markers[unknown==255] = 0

markers = cv2.watershed(img,markers)
plt.imshow(markers, cmap='jet')
# plt.show()
img[markers == -1] = [255,0,0]

plt.imshow(img, cmap='gray')
plt.show()

In [ ]:
%matplotlib qt

import skimage 
from skimage import exposure
from skimage import morphology
from scipy import ndimage
from skimage import measure,morphology
import re

src_fixed = r'W:\Vardan_Andriasyan\180910-VA-WT-TC18-IMAGED-FIX\TimePoint_1\180910-VA-WT-TC18-IMAGED-FIX_B02_s10_w1.tif'
src_nfixed = r'W:\Vardan_Andriasyan\180307-VA-dE3B-TC-7\TimePoint_26\180307-VA-dE3B-TC-7_B02_s10_w1.tif'
TESTIM = r'W:\Vardan_Andriasyan\POSTPROCESSING\racaniello_pres\res\theshold_me.tif'


inputPath = '-'

#TESTIM = 'test_image.tif'




inputIm =  cv2.imread(src_nfixed,-1)

x=0
y=0
h=500
w=500

procIm  = inputIm[y:y+h, x:x+w]

print(procIm.shape)
procIm = exposure.equalize_adapthist(procIm,clip_limit=0.01)
procIm = skimage.util.img_as_ubyte(procIm)

# binary_global = cv2.adaptiveThreshold(procIm,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,401,0)
# global_thresh = skimage.filters.threshold_local(procIm, 11)#(procIm)#threshold_otsu(procIm)#,

global_thresh = skimage.filters.threshold_otsu(procIm)
print(global_thresh)
binary_global = procIm > global_thresh*0.3
# binary_global = procIm > global_thresh*0.9
print(binary_global)
# binary_global = ndimage.binary_fill_holes(binary_global)
thresholdedImage = binary_global.astype('uint8')
# thresholdedImage = binary_global
# print(thresholdedImage)
# plt.figure(1)
# plt.imshow(thresholdedImage,cmap = 'gray',vmin = 0 ,vmax = 1)
# plt.show()  

minArea = 1000
dilationDisk = 4
selem = skimage.morphology.disk(dilationDisk)


thresholdedImage = cv2.erode(thresholdedImage,selem,iterations = 2)    


labeledImage = measure.label(thresholdedImage)

labeledImage = skimage.morphology.remove_small_objects(labeledImage,min_size = 100)

# Show the segmentations.
fig, axes = plt.subplots(ncols=3, nrows=2, figsize=(9, 5),
                         sharex=True, sharey=True)
ax = axes.ravel()
ax[0].imshow(procIm, cmap='gray')
ax[0].set_title('input Image')


ax[1].imshow(thresholdedImage)
ax[1].set_title('Thresholded Image')

color1 = skimage.color.label2rgb(labeledImage, image=procIm, bg_label=0, image_alpha  = 1)
ax[2].imshow(color1)
ax[2].set_title('Overlay Labels')



for a in ax:
    a.axis('off')
fig.tight_layout()
plt.show()





In [ ]:
import keras

print(keras.__version__)


In [ ]:
import file_utils


dest = r'W:\Vardan_Andriasyan\180629-MB-VA-A549-HSV'


dirList,fileNamelist,fullPathList,fileNum = file_utils.listFilesAndCount(dest)

token_spec = [(r'\\(?P<timepoint>TimePoint_\d*)\\(?!Stitched)'),   
    (r'(?P<well>[A-Z][0-1][1-9])'),         
    (r'_(?P<site>s[0-9]+)'), 
    (r'(?P<channel>w[0-9])_thumb')]


matchedFileList, timepointList, wellList, channelList,siteList  = file_utils.parseImageFileNames(fullPathList,token_specification = token_spec)

print(channelList)


In [ ]:
import file_utils
# filters = [timepointList[0],wellList[0],channelList[0]]
# filteredList = file_utils.filterFileList(matchedFileList,filters)
# print(filteredList)
import importlib

importlib.reload(file_utils)

over = file_utils.generateOverviews (matchedFileList,timepointList, wellList, channelList, siteList,
                          overRows=3, overCols=6, overResizeFactor = 1, overPadding=30,
                          siteRows = 7, siteCols = 7, 
                          outputFilename = r'W:\Vardan_Andriasyan\POSTPROCESSING\Overviews\180629-MB-VA-A549-HSV.tif',fontScale = 0.3)





In [ ]:
def segmentSpheroid(inputImagePath,dilationDisk = 12,blockSize = 501, minSpheroidArea = 100):
    ######### SEGMENT SPHEROID #########
        #inputImagePath = "Z:\\Vardan_Andriasyan\\Morphosphere\\HT-29\\Test\\NonSphHealthy\\20160126-corning-all-spheroids-p2-004hps_D03_w1.TIF"
        #dilationDisk = 10
        #blockSize = 501
        #minSpheroidArea = 10000
        #start = time.clock()

    import skimage
    import math
    import cv2
    from scipy import ndimage
    from skimage import measure,morphology
    import matplotlib.pyplot as plt
    import numpy as np

    inputImage = cv2.imread(inputImagePath, -1) #0 = grey, 1=RGB, .. -1 = as is
    
    processedImage = cv2.convertScaleAbs(inputImage, alpha=(255.0/65535.0))
    #processedImage = inputImage.astype('uint8')  #convert to unsigned 8bit
    #processedImage = skimage.util.img_as_ubyte(inputImage) #
    #processedImage = inputImage #don't convert, then doesn't work
    
    # here error for conversion
    thresholdedImage = cv2.adaptiveThreshold(processedImage,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,blockSize,0) #return inverted threshold, 0 = threshold correction factor
    selem = skimage.morphology.disk(dilationDisk)

    thresholdedImage = cv2.dilate(thresholdedImage,selem,iterations = 1)

    thresholdedImage = ndimage.binary_fill_holes(thresholdedImage)

    labeledImage = measure.label(thresholdedImage)
    allProperties = measure.regionprops(labeledImage)


    imageHeight, imageWidth = thresholdedImage.shape[:2]
    #initialize empty arrays for area and distance filtering
    areas=np.empty(len(allProperties))
    distancesToCenter = np.empty(len(allProperties))
    labels = np.empty(len(allProperties))

    #find the index connected area which is closest to center of the image also area filter
    i = 0
    for props in allProperties:
        y0, x0 = props.centroid
        distance = math.sqrt((y0 -imageHeight/2)**2 + (x0 -imageWidth/2)**2)
        distancesToCenter[i] = distance
        areas[i] = props.area
        labels[i] = props.label
        i=i+1

    #filter by area
    distancesToCenter = distancesToCenter[areas>minSpheroidArea]
    labels = labels[areas>minSpheroidArea]
    # filter by distance and get the index
    indexOfMinDistance = labels[distancesToCenter == min(distancesToCenter)]
    indexOfMinDistance = indexOfMinDistance.astype(int) -1

    selectedCC = (labeledImage == allProperties[indexOfMinDistance].label)

    spheroidBWImage = selectedCC.astype("uint8")

    boundingBox =  allProperties[indexOfMinDistance].bbox

    #get all geometric measurementsi
    centroid = allProperties[indexOfMinDistance].centroid
    perimeter = allProperties[indexOfMinDistance].perimeter
    area = allProperties[indexOfMinDistance].area
    diameter = allProperties[indexOfMinDistance].equivalent_diameter
    majorAxis = allProperties[indexOfMinDistance].major_axis_length
    minorAxis = allProperties[indexOfMinDistance].minor_axis_length
    circularity = 4*math.pi*(area/perimeter**2)

    # Make output Images square

    outputImageSide = max(boundingBox[2]-boundingBox[0],boundingBox[3]-boundingBox[1])

    minRow = round(centroid[0]-outputImageSide/2)
    maxRow = round(centroid[0]+outputImageSide/2)
    minCol = round(centroid[1]-outputImageSide/2)
    maxCol = round(centroid[1]+outputImageSide/2)

    croppedBWImage = spheroidBWImage[minRow:maxRow,minCol:maxCol]
    croppedImage  =  inputImage[minRow:maxRow,minCol:maxCol]*croppedBWImage

    spheroidAttributes = {'area': area ,'diameter': diameter,'circularity': circularity,'majorAxis': majorAxis,'minorAxis': minorAxis}

    fullImage = inputImage*spheroidBWImage

    plt.figure(1)
    plt.imshow(croppedBWImage)
    plt.figure(3)
    plt.imshow(croppedImage)

    # plot_comparison(inputImage,spheroidBWImage,inputImagePath)
    plt.set_cmap('gray')
    plt.show()

   
    return croppedImage,croppedBWImage,spheroidAttributes

# Segmentation Snippets


In [ ]:
%matplotlib qt

In [ ]:
%matplotlib qt

import tifffile as TIF
from skimage import io
from skimage import feature
from skimage import exposure
from skimage import morphology
from scipy import ndimage
from skimage import measure,morphology
import re


from skimage.filters import sobel
import numpy as np
import cv2
from matplotlib import pyplot as plt



nucImNPath = r'W:\Vardan_Andriasyan\PAPER\new_figs\RES\SEGM\hsv_N.tif'
nucImVPath = r'W:\Vardan_Andriasyan\PAPER\new_figs\RES\SEGM\hsv_V.tif'

inputVim = cv2.imread(nucImVPath,-1)

inputIm =  cv2.imread(nucImNPath,-1)
# inputIm =  .imread(ex)

# inputIm = cv2.convertScaleAbs(inputIm, alpha=(255.0/65535.0))

# clahe = cv2.createCLAHE(clipLimit=0.02, tileGridSize=(8,8))
# inputIm = clahe.apply(inputIm)    

# inputIm = exposure.equalize_adapthist(inputIm,clip_limit=0.2)
# inputIm = skimage.util.img_as_ubyte(inputIm)


fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(inputIm, cmap=plt.cm.gray, interpolation='nearest',vmin=0,vmax =50)
ax.set_title('input image')
ax.axis('off')

# inputIm = skimage.filters.gaussian(inputIm,sigma=3,preserve_range = True)
# # diskSize = 25
# # s=skimage.morphology.disk(diskSize)
# # inputIm = skimage.morphology.opening(inputIm,selem=s)
# # inputIm = skimage.morphology.white_tophat(inputIm, selem=s)
# # inputIm = skimage.util.invert(skimage.morphology.white_tophat(inputIm, selem=s)-inputIm)
    
# fig, ax = plt.subplots(figsize=(20, 10))
# ax.imshow(inputIm, cmap=plt.cm.gray, interpolation='nearest',vmin=0,vmax =50)
# ax.set_title('input image')
# ax.axis('off')

segm = inputIm >6

segm = skimage.morphology.binary_closing(segm,skimage.morphology.disk(1))


segm = skimage.morphology.remove_small_objects(segm,min_size = 100)
# segm = morphology.misc.remove_small_holes(segm)


# segm = ndimage.binary_fill_holes(segm)    


# segm = ndimage.binary_fill_holes(th2)
# th2 = ndimage.binary_fill_holes(th2)

# fig, ax = skimage.filters.try_all_threshold(inputIm, figsize=(20, 10), verbose=False)
# plt.show()


# block_size = 451
# segm = skimage.filters.threshold_adaptive(inputIm, block_size, offset=0)


# # # segm = skimage.morphology.binary_dilation(segm,skimage.morphology.disk(3))


fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(segm, cmap=plt.cm.nipy_spectral, interpolation='nearest')
ax.set_title('markers')
ax.axis('off')


distance = ndimage.distance_transform_edt(segm) 
# distance = skimage.filters.gaussian(distance,sigma=4)

fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(distance, cmap=plt.cm.nipy_spectral, interpolation='nearest')
ax.set_title('markers')
ax.axis('off')


maxRegSize = 4

distCell = 30

peaks = skimage.feature.peak_local_max(distance, indices=False, footprint=np.ones((distCell,distCell)),labels=segm)

selem = skimage.morphology.disk(maxRegSize)
peaks = skimage.morphology.binary_dilation(peaks,selem)


fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(peaks , cmap=plt.cm.nipy_spectral, interpolation='nearest')
ax.set_title('markers')
ax.axis('off')


markers = ndimage.label(peaks)[0]
labels = skimage.segmentation.watershed(-distance, markers, mask=segm,watershed_line= True )
labels = skimage.morphology.remove_small_objects(labels,min_size = 750)


labels = ndimage.label(labels)[0]


# labels = skimage.segmentation.clear_border(labels)

fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(labels, cmap=plt.cm.nipy_spectral, interpolation='nearest')
ax.set_title('markers')
ax.axis('off')

color1 = skimage.color.label2rgb(labels, image=inputIm, bg_label=0, image_alpha  = 1)
fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(color1, cmap=plt.cm.nipy_spectral, interpolation='nearest',vmin=0,vmax=50)
ax.set_title('markers')
ax.axis('off')


print(inputVim)
virBW = inputVim > 1450

virBW = skimage.morphology.remove_small_objects(virBW,min_size = 300)
virBW = skimage.morphology.remove_small_objects(virBW,min_size = 300)

virBW = skimage.morphology.binary_closing(virBW,skimage.morphology.disk(9))
# virBW = skimage.morphology.binary_erosion(virBW,selem = skimage.morphology.disk(6))




fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(virBW)
ax.set_title('markers')
ax.axis('off')

nucBW =  labels >0

fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(nucBW)
ax.set_title('markers')
ax.axis('off')



newLabels = labels

iLabel = 1




for region in skimage.measure.regionprops(newLabels,cache=False):
  
    largeBBox = np.full(labels.shape,False,dtype=bool)
    (x1,y1,x2,y2) = region.bbox
    largeBBox[x1:x2,y1:y2] = region.filled_image
    curVirBW = virBW*largeBBox
# print(curVirBW.sum())
    if curVirBW.sum() > 0:
        newLabels[newLabels == iLabel] = 60
    else:
        newLabels[newLabels == iLabel] = 128
        
    print(iLabel)
    
    iLabel=iLabel+1    
  

        
        
        
color1 = skimage.color.label2rgb(newLabels,bg_label=0)   
fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(color1, cmap=plt.cm.jet, interpolation='nearest')
ax.set_title('relabelled')
ax.axis('off')
        
        
skimage.io.imsave(r'W:\Vardan_Andriasyan\PAPER\new_figs\RES\SEGM\HSV_pred.png',color1)
    
    
    



# L = skimage.morphology.binary_closing(L,skimage.morphology.disk(2))

# L = bwlabel(BW_v);
# L = L+BW_n;
# L(L >1) = 200;

    

    
    

# ADENO dE3B


In [1]:
%matplotlib qt

import tifffile as TIF
import skimage
from skimage import io
from skimage import feature
from skimage import exposure
from skimage import morphology
from scipy import ndimage
from skimage import measure,morphology
import re


from skimage.filters import sobel
import numpy as np
import cv2
from matplotlib import pyplot as plt



nucImNPath = r'W:\Vardan_Andriasyan\PAPER\new_figs\RES\SEGM\de3b_N.tif'
nucImVPath = r'W:\Vardan_Andriasyan\PAPER\new_figs\RES\SEGM\de3b_V.tif'

inputVim = cv2.imread(nucImVPath,-1)

inputIm =  cv2.imread(nucImNPath,-1)

clahe = cv2.createCLAHE(clipLimit=0.1, tileGridSize=(30,30))
inputIm = clahe.apply(inputIm)

fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(inputIm, cmap=plt.cm.gray, interpolation='nearest',vmin=0,vmax =1000)
ax.set_title('input image')
ax.axis('off')

# inputIm = skimage.filters.gaussian(inputIm,sigma=30,preserve_range = True)
diskSize = 50
s=skimage.morphology.disk(diskSize)
# inputIm = skimage.morphology.opening(inputIm,selem=s)
inputIm = skimage.morphology.white_tophat(inputIm, selem=s)
# # inputIm = skimage.util.invert(skimage.morphology.white_tophat(inputIm, selem=s)-inputIm)
    
fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(inputIm, cmap=plt.cm.gray, interpolation='nearest',vmin=0,vmax =1000)
ax.set_title('input image')
ax.axis('off')

segm = inputIm >90
# fig, ax = skimage.filters.try_all_threshold(inputIm, figsize=(20, 10), verbose=False)
# plt.show()

segm = skimage.morphology.binary_closing(segm,skimage.morphology.disk(1))


segm = skimage.morphology.remove_small_objects(segm,min_size = 100)
segm = morphology.misc.remove_small_holes(segm)


segm = ndimage.binary_fill_holes(segm)    


# segm = ndimage.binary_fill_holes(th2)
# th2 = ndimage.binary_fill_holes(th2)

# fig, ax = skimage.filters.try_all_threshold(inputIm, figsize=(20, 10), verbose=False)
# plt.show()


# block_size = 451
# segm = skimage.filters.threshold_adaptive(inputIm, block_size, offset=0)


# # # segm = skimage.morphology.binary_dilation(segm,skimage.morphology.disk(3))


fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(segm, cmap=plt.cm.nipy_spectral, interpolation='nearest')
ax.set_title('markers')
ax.axis('off')


distance = ndimage.distance_transform_edt(segm) 
# distance = skimage.filters.gaussian(distance,sigma=4)

fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(distance, cmap=plt.cm.nipy_spectral, interpolation='nearest')
ax.set_title('markers')
ax.axis('off')


maxRegSize = 4

distCell = 30

peaks = skimage.feature.peak_local_max(distance, indic1es=False, footprint=np.ones((distCell,distCell)),labels=segm)

selem = skimage.morphology.disk(maxRegSize)
peaks = skimage.morphology.binary_dilation(peaks,selem)


fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(peaks , cmap=plt.cm.nipy_spectral, interpolation='nearest')
ax.set_title('markers')
ax.axis('off')


markers = ndimage.label(peaks)[0]
labels = skimage.segmentation.watershed(-distance, markers, mask=segm,watershed_line= True )
labels = skimage.morphology.remove_small_objects(labels,min_size = 750)


labels = ndimage.label(labels)[0]


# labels = skimage.segmentation.clear_border(labels)

fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(labels, interpolation='nearest')
#ax.set_title('markers')
ax.axis('off')

color1 = skimage.color.label2rgb(labels, image=inputIm, bg_label=0, image_alpha  = 1)
fig, ax = plt.subplots(figsize=(20, 10))b 
ax.imshow(color1, cmap=plt.cm.nipy_spectral, interpolation='nearest',vmin=0,vmax=50)
ax.set_title('markers')
ax.axis('off')


print(inputVim)


virBW = inputVim > 1450

fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(inputVim,vmin=0,vmax=2000)
ax.set_title('markers')
ax.axis('off')

virBW = skimage.morphology.remove_small_objects(virBW,min_size = 300)

virBW = skimage.morphology.binary_closing(virBW,skimage.morphology.disk(9))
# virBW = skimage.morphology.binary_erosion(virBW,selem = skimage.morphology.disk(6))




fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(virBW)
ax.set_title('markers')
ax.axis('off')

nucBW =  labels >0

fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(nucBW)
ax.set_title('markers')
ax.axis('off')



newLabels = labels

iLabel = 1




for region in skimage.measure.regionprops(newLabels,cache=False):
  
    largeBBox = np.full(labels.shape,False,dtype=bool)
    (x1,y1,x2,y2) = region.bbox
    largeBBox[x1:x2,y1:y2] = region.filled_image
    curVirBW = virBW*largeBBox
# print(curVirBW.sum())
    if curVirBW.sum() > 0:
        newLabels[newLabels == iLabel] = 60
    else:
        newLabels[newLabels == iLabel] = 128
        
    
    
    iLabel=iLabel+1    
  

        
        
        
color1 = skimage.color.label2rgb(newLabels,bg_label=0)   
fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(color1, cmap=plt.cm.jet, interpolation='nearest')
ax.set_title('relabelled')
ax.axis('off')
        
        
# skimage.io.imsave(r'W:\Vardan_Andriasyan\PAPER\new_figs\RES\SEGM\dE3B_pred.png',color1)
    

C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\skimage\util\arraycrop.py:175: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  cropped = np.array(ar[slices], order=order, copy=True)


[[795 770 684 ... 731 691 698]
 [714 712 702 ... 707 699 715]
 [728 716 723 ... 705 717 696]
 ...
 [711 724 730 ... 824 747 767]
 [710 695 689 ... 801 828 777]
 [729 734 698 ... 762 765 745]]


(-0.5, 1958.5, 1958.5, -0.5)

# ADENO fixed


In [4]:
%matplotlib qt

import skimage
import tifffile as TIF
from skimage import io
from skimage import feature
from skimage import exposure
from skimage import morphology
from scipy import ndimage
from skimage import measure,morphology
import re


from skimage.filters import sobel
import numpy as np
import cv2
from matplotlib import pyplot as plt



nucImNPath = r'W:\Vardan_Andriasyan\PAPER\new_figs\RES\SEGM\wt_N.tif'
nucImVPath = r'W:\Vardan_Andriasyan\PAPER\new_figs\RES\SEGM\wt_V.tif'

inputVim = cv2.imread(nucImVPath,-1)

inputIm =  cv2.imread(nucImNPath,-1)

clahe = cv2.createCLAHE(clipLimit=0.1, tileGridSize=(30,30))
inputIm = clahe.apply(inputIm)

fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(inputIm, cmap=plt.cm.gray, interpolation='nearest',vmin=0,vmax =1000)
ax.set_title('input image')
ax.axis('off')


segm = inputIm >250



fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(segm, cmap=plt.cm.nipy_spectral, interpolation='nearest')
ax.set_title('markers')
ax.axis('off')


distance = ndimage.distance_transform_edt(segm) 
# distance = skimage.filters.gaussian(distance,sigma=4)

fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(distance, cmap=plt.cm.nipy_spectral, interpolation='nearest')
ax.set_title('markers')
ax.axis('off')


maxRegSize = 4

distCell = 30

peaks = skimage.feature.peak_local_max(distance, indices=False, footprint=np.ones((distCell,distCell)),labels=segm)

selem = skimage.morphology.disk(maxRegSize)
peaks = skimage.morphology.binary_dilation(peaks,selem)


fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(peaks , cmap=plt.cm.nipy_spectral, interpolation='nearest')
ax.set_title('markers')
ax.axis('off')


markers = ndimage.label(peaks)[0]
labels = skimage.segmentation.watershed(-distance, markers, mask=segm,watershed_line= True )
labels = skimage.morphology.remove_small_objects(labels,min_size = 750)


labels = ndimage.label(labels)[0]


# # labels = skimage.segmentation.clear_border(labels)

fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(labels, interpolation='nearest')
ax.set_title('markers')
ax.axis('off')

color1 = skimage.color.label2rgb(labels, image=inputIm, bg_label=0, image_alpha  = 1)
fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(color1, cmap=plt.cm.nipy_spectral, interpolation='nearest',vmin=0,vmax=50)
ax.set_title('markers')
ax.axis('off')


# print(inputVim)


virBW = inputVim > 390

fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(inputVim,vmin=0,vmax=2000)
ax.set_title('markers')
ax.axis('off')

virBW = skimage.morphology.remove_small_objects(virBW,min_size = 300)


virBW = skimage.morphology.binary_closing(virBW,skimage.morphology.disk(9))
# virBW = skimage.morphology.binary_erosion(virBW,selem = skimage.morphology.disk(6))




fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(virBW)
ax.set_title('markers')
ax.axis('off')

nucBW =  labels >0

fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(nucBW)
ax.set_title('markers')
ax.axis('off')



newLabels = labels

iLabel = 1




for region in skimage.measure.regionprops(newLabels,cache=False):
  
    largeBBox = np.full(labels.shape,False,dtype=bool)
    (x1,y1,x2,y2) = region.bbox
    largeBBox[x1:x2,y1:y2] = region.filled_image
    curVirBW = virBW*largeBBox
# print(curVirBW.sum())
    if curVirBW.sum() > 0:
        newLabels[newLabels == iLabel] = 60
    else:
        newLabels[newLabels == iLabel] = 128
        
    
    
    iLabel=iLabel+1    
  

        
        
        
        
color1 = skimage.color.label2rgb(newLabels,bg_label=0)   
fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(color1, cmap=plt.cm.jet, interpolation='nearest')
ax.set_title('relabelled')
ax.axis('off')
        
        
skimage.io.imsave(r'W:\Vardan_Andriasyan\PAPER\new_figs\RES\SEGM\wt_pred.png',color1)

C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\skimage\util\arraycrop.py:175: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  cropped = np.array(ar[slices], order=order, copy=True)
C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\skimage\util\dtype.py:130: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


# no infection

In [4]:
%matplotlib inline

import tifffile as TIF
from skimage import io
from skimage import feature
from skimage import exposure
from skimage import morphology
from scipy import ndimage
from skimage import measure,morphology
import re


from skimage.filters import sobel
import numpy as np
import cv2
from matplotlib import pyplot as plt



nucImNPath = r'W:\Vardan_Andriasyan\PAPER\new_figs\RES\SEGM\noInf_N.tif'
nucImVPath = r'W:\Vardan_Andriasyan\PAPER\new_figs\RES\SEGM\noInf_V.tif'

inputVim = cv2.imread(nucImVPath,-1)

inputIm =  cv2.imread(nucImNPath,-1)

clahe = cv2.createCLAHE(clipLimit=0.1, tileGridSize=(30,30))
inputIm = clahe.apply(inputIm)

fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(inputIm, cmap=plt.cm.gray, interpolation='nearest',vmin=0,vmax =1000)
ax.set_title('input image')
ax.axis('off')

# inputIm = skimage.filters.gaussian(inputIm,sigma=30,preserve_range = True)
diskSize = 50
s=skimage.morphology.disk(diskSize)
# inputIm = skimage.morphology.opening(inputIm,selem=s)
inputIm = skimage.morphology.white_tophat(inputIm, selem=s)
# # inputIm = skimage.util.invert(skimage.morphology.white_tophat(inputIm, selem=s)-inputIm)
    
fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(inputIm, cmap=plt.cm.gray, interpolation='nearest',vmin=0,vmax =1000)
ax.set_title('input image')
ax.axis('off')

segm = inputIm >250
# fig, ax = skimage.filters.try_all_threshold(inputIm, figsize=(20, 10), verbose=False)
# plt.show()

segm = skimage.morphology.binary_closing(segm,skimage.morphology.disk(1))


segm = skimage.morphology.remove_small_objects(segm,min_size = 100)
segm = morphology.misc.remove_small_holes(segm)


segm = ndimage.binary_fill_holes(segm)    


# segm = ndimage.binary_fill_holes(th2)
# th2 = ndimage.binary_fill_holes(th2)

# fig, ax = skimage.filters.try_all_threshold(inputIm, figsize=(20, 10), verbose=False)
# plt.show()


# block_size = 451
# segm = skimage.filters.threshold_adaptive(inputIm, block_size, offset=0)


# # # segm = skimage.morphology.binary_dilation(segm,skimage.morphology.disk(3))


fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(segm, cmap=plt.cm.nipy_spectral, interpolation='nearest')
ax.set_title('markers')
ax.axis('off')


distance = ndimage.distance_transform_edt(segm) 
# distance = skimage.filters.gaussian(distance,sigma=4)

fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(distance, cmap=plt.cm.nipy_spectral, interpolation='nearest')
ax.set_title('markers')
ax.axis('off')


maxRegSize = 4

distCell = 30

peaks = skimage.feature.peak_local_max(distance, indices=False, footprint=np.ones((distCell,distCell)),labels=segm)

selem = skimage.morphology.disk(maxRegSize)
peaks = skimage.morphology.binary_dilation(peaks,selem)


fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(peaks , cmap=plt.cm.nipy_spectral, interpolation='nearest')
ax.set_title('markers')
ax.axis('off')


markers = ndimage.label(peaks)[0]
labels = skimage.segmentation.watershed(-distance, markers, mask=segm,watershed_line= True )
labels = skimage.morphology.remove_small_objects(labels,min_size = 750)


labels = ndimage.label(labels)[0]


# labels = skimage.segmentation.clear_border(labels)

fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(labels, cmap=plt.cm.nipy_spectral, interpolation='nearest')
ax.set_title('markers')
ax.axis('off')

color1 = skimage.color.label2rgb(labels, image=inputIm, bg_label=0, image_alpha  = 1)
fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(color1, cmap=plt.cm.nipy_spectral, interpolation='nearest',vmin=0,vmax=50)
ax.set_title('markers')
ax.axis('off')


print(inputVim)


virBW = inputVim > 3000

fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(inputVim,vmin=0,vmax=2000)
ax.set_title('markers')
ax.axis('off')

virBW = skimage.morphology.remove_small_objects(virBW,min_size = 300)
virBW = skimage.morphology.remove_small_objects(virBW,min_size = 300)

virBW = skimage.morphology.binary_closing(virBW,skimage.morphology.disk(9))
# virBW = skimage.morphology.binary_erosion(virBW,selem = skimage.morphology.disk(6))




fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(virBW)
ax.set_title('markers')
ax.axis('off')

nucBW =  labels >0

fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(nucBW)
ax.set_title('markers')
ax.axis('off')



newLabels = labels

iLabel = 1




for region in skimage.measure.regionprops(newLabels,cache=False):
  
    largeBBox = np.full(labels.shape,False,dtype=bool)
    (x1,y1,x2,y2) = region.bbox
    largeBBox[x1:x2,y1:y2] = region.filled_image
    curVirBW = virBW*largeBBox
# print(curVirBW.sum())
    if curVirBW.sum() > 0:
        newLabels[newLabels == iLabel] = 60
    else:
        newLabels[newLabels == iLabel] = 128
        
    
    
    iLabel=iLabel+1    
  

        
        
        
color1 = skimage.color.label2rgb(newLabels,colors = [(0,0,1)], bg_label=0)   
fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(color1, cmap=plt.cm.jet, interpolation='nearest')
ax.set_title('relabelled')
ax.axis('off')
        
        
skimage.io.imsave(r'W:\Vardan_Andriasyan\PAPER\new_figs\RES\SEGM\noInf_pred.png',color1)

TypeError: Image data cannot be converted to float

In [ ]:
%matplotlib qt

import skimage
import tifffile as TIF
from skimage import io
from skimage import feature
from skimage import exposure
from skimage import morphology
from scipy import ndimage
from skimage import measure,morphology
import re


from skimage.filters import sobel
import numpy as np
import cv2
from matplotlib import pyplot as plt




de3b = r'W:\Vardan_Andriasyan\PAPER\new_figs\RES\ROI\d3EB_s_N.png'
wt = r'W:\Vardan_Andriasyan\PAPER\new_figs\RES\ROI\wt_s_N.png'
noInf = r'W:\Vardan_Andriasyan\PAPER\new_figs\RES\ROI\nonInf_s_N.png'
hsv = r'W:\Vardan_Andriasyan\PAPER\new_figs\RES\ROI\hsv_s_N.png'



inputIm =  cv2.imread(de3b,-1)
inputIm = skimage.color.rgb2gray(inputIm)*255
fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(inputIm, cmap=plt.cm.gray, interpolation='nearest',vmin=0,vmax =150)
ax.set_title('input image')
ax.axis('off')

segm = inputIm > 60
segm = skimage.morphology.binary_dilation(segm,skimage.morphology.disk(3))
segm = skimage.morphology.remove_small_objects(segm,min_size = 100)
segm = morphology.misc.remove_small_holes(segm)
segm = ndimage.binary_fill_holes(segm)    

fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(segm, cmap=plt.cm.nipy_spectral, interpolation='nearest')
ax.set_title('markers')
ax.axis('off')

        
color1 = skimage.color.label2rgb(segm,colors = [(1,0,0)], bg_label=0)   
        
        
skimage.io.imsave(r'W:\Vardan_Andriasyan\PAPER\new_figs\RES\SEGM\de3b_s.png',color1)

#############################################
inputIm =  cv2.imread(wt,-1)
inputIm = skimage.color.rgb2gray(inputIm)*255
fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(inputIm, cmap=plt.cm.gray, interpolation='nearest',vmin=0,vmax =150)
ax.set_title('input image')
ax.axis('off')

segm = inputIm > 60

segm = skimage.morphology.binary_dilation(segm,skimage.morphology.disk(3))
segm = skimage.morphology.remove_small_objects(segm,min_size = 100)
segm = morphology.misc.remove_small_holes(segm)
segm = ndimage.binary_fill_holes(segm)    

fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(segm, cmap=plt.cm.nipy_spectral, interpolation='nearest')
ax.set_title('markers')
ax.axis('off')


color1 = skimage.color.label2rgb(segm,colors = [(1,0,0)], bg_label=0)   
        
        
skimage.io.imsave(r'W:\Vardan_Andriasyan\PAPER\new_figs\RES\SEGM\wt_s.png',color1)


#############################################
inputIm =  cv2.imread(hsv,-1)
inputIm = skimage.color.rgb2gray(inputIm)*255
fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(inputIm, cmap=plt.cm.gray, interpolation='nearest',vmin=0,vmax =150)
ax.set_title('input image')
ax.axis('off')

segm = inputIm > 60

segm = skimage.morphology.binary_dilation(segm,skimage.morphology.disk(3))
segm = skimage.morphology.remove_small_objects(segm,min_size = 100)
segm = morphology.misc.remove_small_holes(segm)
segm = ndimage.binary_fill_holes(segm)    

fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(segm, cmap=plt.cm.nipy_spectral, interpolation='nearest')
ax.set_title('markers')
ax.axis('off')

color1 = skimage.color.label2rgb(segm,colors = [(1,0,0)], bg_label=0)   
        
        
skimage.io.imsave(r'W:\Vardan_Andriasyan\PAPER\new_figs\RES\SEGM\hsv_s.png',color1)


#############################################


#############################################
inputIm =  cv2.imread(noInf,-1)
inputIm = skimage.color.rgb2gray(inputIm)*255
fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(inputIm, cmap=plt.cm.gray, interpolation='nearest',vmin=0,vmax =150)
ax.set_title('input image')
ax.axis('off')

segm = inputIm > 60

segm = skimage.morphology.binary_dilation(segm,skimage.morphology.disk(3))
segm = skimage.morphology.remove_small_objects(segm,min_size = 100)
segm = morphology.misc.remove_small_holes(segm)
segm = ndimage.binary_fill_holes(segm)    

fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(segm, cmap=plt.cm.nipy_spectral, interpolation='nearest')
ax.set_title('markers')
ax.axis('off')

color1 = skimage.color.label2rgb(segm,colors = [(0,0,1)], bg_label=0)   
        
        
skimage.io.imsave(r'W:\Vardan_Andriasyan\PAPER\new_figs\RES\SEGM\noInf_s.png',color1)

#############################################



In [ ]:
segm = inputIm >90 1
# segm = morphology.misc.remove_small_holes(segm)
# fig, ax = skimage.filters.try_all_threshold(inputIm, figsize=(20, 10), verbose=False)
# plt.show()

# segm = skimage.morphology.binary_closing(segm,skimage.morphology.disk(1))


# segm = skimage.morphology.remove_small_objects(segm,min_size = 100)
# segm = morphology.misc.remove_small_holes(segm)


# segm = ndimage.binary_fill_holes(segm)    


# # segm = ndimage.binary_fill_holes(th2)
# # th2 = ndimage.binary_fill_holes(th2)

# # fig, ax = skimage.filters.try_all_threshold(inputIm, figsize=(20, 10), verbose=False)
# # plt.show()


# # block_size = 451
# # segm = skimage.filters.threshold_adaptive(inputIm, block_size, offset=0)


# # # # segm = skimage.morphology.binary_dilation(segm,skimage.morphology.disk(3))


fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(segm, cmap=plt.cm.nipy_spectral, interpolation='nearest')
ax.set_title('markers')
ax.axis('off')

In [ ]:

inputIm =  cv2.imread(r'W:\Vardan_Andriasyan\PAPER\new_figs\RES\SEGM\testIm.tif',-1)

bw = inputIm > 10

print(bw)

labels = ndimage.label(bw)[0]

iLabel = 1
for region in regionprops(labels):
    print(iLabel)
            
    iLabel = iLabel + 1 
ax.imshow(newLabels, cmap=plt.cm.jet, interpolation='nearest')
ax.set_title('markers')
ax.axis('off')
        

# inputIm =  .imread(ex)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi

from skimage.morphology import watershed
from skimage.feature import peak_local_max


# Generate an initial image with two overlapping circles
x, y = np.indices((80, 80))
x1, y1, x2, y2 = 28, 28, 44, 52
r1, r2 = 16, 20
mask_circle1 = (x - x1)**2 + (y - y1)**2 < r1**2
mask_circle2 = (x - x2)**2 + (y - y2)**2 < r2**2
image = np.logical_or(mask_circle1, mask_circle2)
print(image)
image = skimage.img_as_bool(segmentation)
print(segmentation)
# Now we want to separate the two objects in image
# Generate the markers as local maxima of the distance to the background
distance = ndi.distance_transform_edt(image)
local_maxi = peak_local_max(distance, indices=False, footprint=np.ones((3, 3)),
                            labels=image)
markers = ndi.label(local_maxi)[0]
labels = watershed(-distance, markers, mask=image)

fig, axes = plt.subplots(ncols=3, figsize=(9, 3), sharex=True, sharey=True)
ax = axes.ravel()

ax[0].imshow(image, cmap=plt.cm.gray, interpolation='nearest')
ax[0].set_title('Overlapping objects')
ax[1].imshow(-distance, cmap=plt.cm.gray, interpolation='nearest')
ax[1].set_title('Distances')
ax[2].imshow(labels, cmap=plt.cm.nipy_spectral, interpolation='nearest')
ax[2].set_title('Separated objects')

for a in ax:
    a.set_axis_off()

fig.tight_layout()
plt.show()

In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt





ex = r'W:\Vardan_Andriasyan\PAPER\new_figs\RES\SEGM\hsv_N.tif'

img =  cv2.imread(ex,-1)

# img = cv2.imread('coins.png')
# gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(img,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
print(segmentation*255)
cv2.imshow('image',thresh)
cv2.waitKey(0)

In [ ]:



# elevation_map = sobel(inputIm)




# inputIm = skimage.filters.gaussian(inputIm,sigma=3,preserve_range = True )
# inputIm = skimage.morphology.black_tophat(inputIm)
# inputIm = skimage.filters.rank.entropy(inputIm, skimage.morphology.disk(1))






# binary_global = ndimage.binary_fill_holes(binary_global)    
# labeledImage = skimage.morphology.remove_small_objects(labeledImage,min_size = 100)






# ax.imshow(segm, cmap=plt.cm.gray, interpolation='nearest')
# ax.set_title('elevation map')
# ax.axis('off')


# skimage.morphology.binary_dilation


# peaks =  skimage.morphology.binary_dilation(peaks )


# peaks =  skimage.morphology.local_maxima(distance) 
# peaks = distance > 20
# print(distance.max())

# image = inputIm

# local_maxi = peak_local_max(distance, indices=False, footprint=np.ones((5,5)))

# markers = ndi.label(local_maxi)[0]

# labels = skimage.(segm, peaks)







# entr_img = skimage.filters.rank.entropy(inputIm, morphology.disk(10))

# ax.imshow(entr_img, cmap=plt.cm.nipy_spectral, interpolation='nearest')
# ax.set_title('markers')
# ax.axis('off')








# from skimage.filters import try_all_threshold







# from skimage.filters import try_all_threshold
# fig, ax = try_all_threshold(entr_img, figsize=(100, 80), verbose=False)
# plt.show()


# markers2 = ndimage.label(local_maxi)[0]

# markers = np.zeros_like(inputIm)
# markers[inputIm < 5] = 1
# markers[inputIm > 6] = 2




# fig, ax = plt.subplots(figsize=(20, 10))
# ax.imshow(markers, cmap=plt.cm.nipy_spectral, interpolation='nearest')
# ax.set_title('markers')
# ax.axis('off')



# segmentation = morphology.watershed(elevation_map, markers)


# print(segmentation.max())
# fig, ax = plt.subplots(figsize=(20, 10))
# ax.imshow(segmentation, cmap=plt.cm.gray, interpolation='nearest')
# ax.set_title('segmentation')
# ax.axis('off')


                                                                                                                                      

# # ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

# distance = ndimage.distance_transform_edt(segmentation)

# local_maxi = skimage.feature.peak_local_max(distance, indices=False, footprint=np.ones((30, 30)),
#                             labels=segmentation)
# markers = ndimage.label(local_maxi)[0]

# fig, ax = plt.subplots(figsize=(20, 10))
# ax.imshow(distance, cmap=plt.cm.gray,vmax = 5000)
# ax.set_title('segmentation')
# ax.axis('off')



# distance = ndimage.distance_transform_edt(segmentation) 
# plt.imshow(distance, cmap=plt.cm.gray, interpolation='nearest')
# local_maxi = skimage.feature.peak_local_max(distance, indices=False, footprint=np.ones((3, 3)),
#                             labels=segmentation)
# markers2 = ndimage.label(local_maxi)[0]


# labels = morphology.watershed(-distance, markers, mask=segmentation)

# fig, ax = plt.subplots(figsize=(20, 10))
# ax.imshow(labels, cmap=plt.cm.gray, interpolation='nearest')
# ax.set_title('segmentation')
# ax.axis('off')


# fig, ax = try_all_threshold(entr_img, figsize=(100, 80), verbose=False)
# plt.show()


# markers2 = ndimage.label(local_maxi)[0]

# markers = np.zeros_like(inputIm)
# markers[inputIm < 5] = 1
# markers[inputIm > 6] = 2




# fig, ax = plt.subplots(figsize=(20, 10))
# ax.imshow(markers, cmap=plt.cm.nipy_spectral, interpolation='nearest')
# ax.set_title('markers')
# ax.axis('off')



# segmentation = morphology.watershed(elevation_map, markers)


# print(segmentation.max())
# fig, ax = plt.subplots(figsize=(20, 10))
# ax.imshow(segmentation, cmap=plt.cm.gray, interpolation='nearest')
# ax.set_title('segmentation')
# ax.axis('off')


                                                                                                                                      

# # ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

# distance = ndimage.distance_transform_edt(segmentation)

# local_maxi = skimage.feature.peak_local_max(distance, indices=False, footprint=np.ones((30, 30)),
#                             labels=segmentation)
# markers = ndimage.label(local_maxi)[0]

# fig, ax = plt.subplots(figsize=(20, 10))
# ax.imshow(distance, cmap=plt.cm.gray,vmax = 5000)
# ax.set_title('segmentation')
# ax.axis('off')



# distance = ndimage.distance_transform_edt(segmentation) 
# plt.imshow(distance, cmap=plt.cm.gray, interpolation='nearest')
# local_maxi = skimage.feature.peak_local_max(distance, indices=False, footprint=np.ones((3, 3)),
#                             labels=segmentation)
# markers2 = ndimage.label(local_maxi)[0]


# labels = morphology.watershed(-distance, markers, mask=segmentation)

# fig, ax = plt.subplots(figsize=(20, 10))
# ax.imshow(labels, cmap=plt.cm.gray, interpolation='nearest')
# ax.set_title('segmentation')
# ax.axis('off')






#################################################################


# plt.figure(2)
# plt.imshow(labeledImage,cmap = 'jet')
# plt.show()  


    

# allProperties = measure.regionprops(labeledImage)

# areas=np.empty(len(allProperties))
# labels = np.empty(len(allProperties))

# # #find the index connected area which is closest to center of the image also area filter
# i = 0
# for props in allProperties:
#     areas[i] = props.area
#     labels[i] = props.label
#     i=i+1

# labelToRemove = labels[areas<minArea]    
# #filter by area

# for iLabel in labelToRemove:
#     index = np.where(labeledImage == iLabel) 
# #     print(s)
#     labeledImage[s] = 0

   
    


# # # s  = labeledImage[labeledImage != 1]

# plt.figure(3)
# plt.imshow(labeledImage)
# plt.show()  

# np.array_equiv(A,B) 

# print(s) #[ labeledImage == 4 ])


# procIm = procIm  = skimage.util.img_as_float(inputIm)

# procIm =exposure.equalize_hist(procIm) 

# print(np.max(inputIm))     

# procIm = rescale_array(procIm)
# procIm = exposure.rescale_intensity(procIm, in_range=(0, 2**16-1),out_range = (0, 255))
# 

# print(procIm.shape)
# procIm = cv2.convertScaleAbs(inputIm, alpha=(255.0/65535.0))
# procIm = skimage.util.img_as_ubyte(procIm)

# # binary_global = cv2.adaptiveThreshold(procIm,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,401,0)
# global_thresh = skimage.filters.threshold_local(procIm, 501)#(procIm)
# binary_global = procIm > global_thresh*0.9

# print(global_thresh)
# print(np.max(inputIm))
# print(np.max(binary_global))
# plt.hist(procIm.ravel(), 256)

# thresholdedImage = binary_global.astype('uint8')

# print(thresholdedImage)

# minArea = 500
# dilationDisk = 10
# selem = skimage.morphology.disk(dilationDisk)

# thresholdedImage = cv2.erode(thresholdedImage,selem,iterations = 1)    

# labeledImage = measure.label(thresholdedImage)

# plt.figure(1)
# plt.imshow(thresholdedImage,cmap = 'gray',vmin = 0 ,vmax = 1)
# plt.show()  



# allProperties = measure.regionprops(labeledImage)

# # imageHeight, imageWidth = thresholdedImage.shape[:2]

# #initialize empty arrays for area and distance filtering
# areas=np.empty(len(allProperties))
# labels = np.empty(len(allProperties))




# # #find the index connected area which is closest to center of the image also area filter
# i = 0
# for props in allProperties:
#     areas[i] = props.area
#     labels[i] = props.label
#     i=i+1
    
    
# print(labels.shape) 
# print(np.median(areas))


 
# labels = labels[areas>minArea]    
# # #filter by area
# # print(np.mean(areas))
# s = labeledImage == 4
# np.array_equiv(A,B) 
# print(s) #[ labeledImage == 4 ])
# print(labels1)
# # # print(labeledImage)
# # # # filter by distance and get the index
# # # # print(labeledImage = )
# # print

# print()
# plt.figure(2)
# # plt.hist(labeledImage.ravel(), 256)
# plt.imshow(thresholdedImage[labels1[1] == labeledImage],cmap = 'gray',vmin = 0 ,vmax = 1)

# plt.show
# print(sss)

# plt.figure(2)
# plt.imshow(sss,cmap = 'gray',vmin = 0 ,vmax = 1)
# plt.show()  


# print(sss)
# plt.figure(1)00\
# # print([areas>minArea])

# plt.imshow()
# plt.show()  

# plt.figure(2)
# plt.imshow(procIm,cmap = 'gray',vmin = 0 ,vmax = 255)
# plt.show()  


# plt.figure(2)
# plt.imshow(procIm,cmap = 'gray',vmin = 0 ,vmax = 255)
# plt.show()  




# procIm = skimage.util.img_as_ubyte(procIm)
# print(np.max(procIm.ravel()))
# procIm = exposure.equalize_adapthist(procIm)
# print(np.max(procIm))
# plt.imshow(procIm,cmap = 'gray',vmin = 0 ,vmax = 255)




# procIm = exposure.rescale_intensity(procIm, in_range=(0, 255))

# procIm = procIm.astype('uint8')
# # procIm  = skimage.util.img_as_float(inputIm)
# # procIm =exposure.equalize_hist(procIm) 

# # procIm = skimage.util.img_as_ubyte(procIm)
# # procIm = exposure.equalize_adapthist(procIm)
# # procIm =exposure.equalize_hist(procIm) 1
# # procIm = cv2.cvtColor(procIm,cv2.COLOR_GRAY2RGB)


# # # Contrast stretching
# # p2, p98 = np.percentile(procIm, (2, 98))
# # procIm = exposure.rescale_intensity(procIm, in_range=(p2, p98))

# # # Equalization


# print(np.median(procIm.ravel()))
# plt.imshow(procIm,cmap = 'gray',vmin = 0 ,vmax = 1)
# # plt.hist(procIm.ravel(), 256)
# plt.show()  

 

# processedImage = cv2.convertScaleAbs(inputIm, alpha=(255.0/65535.0))
# s = skimage.util.img_as_float(inputIm)
# # A = exposure.equalize_adapthist(s ,clip_limit = 0.05)
# A = exposure.equalize_hist(s)


# A = skimage.util.img_as_ubyte(A)
# # s = cv2.equalizeHist(processedImage)

# plt.figure()
# # cv2.IMWRITE_JPEG_CHROMA_QUALITY
# plt.imshow(A,cmap = 'gray',vmin = 0 ,vmax =255)


    
    
# cv2.imwrite('test_8bit.tif', A)  

# ret2,th2 = cv2.threshold(processedImage ,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

# dilationDisk = 5


# selem = morphology.disk(dilationDisk)



# th2 = cv2.dilate(th2,selem,iterations = 1)

# th2 = ndimage.binary_fill_holes(th2)


# plt.imshow(th2,cmap = 'gray')
# plt.show()  
    
    # blur = cv2.GaussianBlur(processedImage,(1,121),0)

# # plt.imshow(blur,cmap = 'gray')
# # plt.hist(processedImage.ravel(), 256)
# plt.imshow(blur,cmap = 'gray',vmin = 0, vmax =3000)
# plt.show()
# th3 = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,41,0)
# # thresholdedImage = cv2.adaptiveThreshold(processedImage,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,512,1)
# # thresh,sss = cv2.tqhreshold(processedImage,5,255,0)
# print(th3)
# plt.imshow(th3,cmap = 'gray')
# # plt.hist(processedImage.ravel(), 256)
# plt.show()
